In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from tqdm.notebook import trange, tqdm

In [2]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
input_files = r'C:\Users\svo0175\Documents\Work\Svetlomet\Hella_06_05_2021\Hrabova_test_long_nove_svetlo\Hrabova_test_long_nove_svetlo_cut_{0}.{1}'
input_files = r'C:\Users\Petr\Documents\svetlomety\data\Hrabova_test_long_nove_svetlo\Hrabova_test_long_nove_svetlo_cut_{0}.{1}'

thresholds_arr = [10, 50, 100, 115]

In [4]:
dfs = {}
for k in thresholds_arr:
    dfs[k] = pd.read_parquet(input_files.format(k, 'parquet.gzip'), engine='pyarrow')

KeyboardInterrupt: 

In [ ]:
df = dfs[50]

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
##### Jelikož je ve všech sloupcích podobná popisná statistika je pro agregaci hranice detekované přechodové hrany zvolen průměr mezi všemi sloupci obrazu.

In [ ]:
df_avg = pd.DataFrame()
df_avg['FrameTimestamp_us'] = df.FrameTimestamp_us
df_avg['avg_edge_occurence'] = df.iloc[:, :-1].mean(axis=1)
print(df_avg.shape)
df_avg.head()

In [ ]:
df_avg.describe()

In [ ]:
#### Zobrazení signálu

In [ ]:
df_tmp = df_avg.iloc[:50000]
px.line(x=df_tmp.FrameTimestamp_us, y=df_tmp.avg_edge_occurence)

In [ ]:
df_tmp = df_avg.iloc[50000:150000]
px.line(x=df_tmp.FrameTimestamp_us, y=df_tmp.avg_edge_occurence)

In [ ]:
df_tmp = df_avg
px.line(x=df_tmp.FrameTimestamp_us, y=df_tmp.avg_edge_occurence)

In [ ]:
#### Kontrola velkých změn mezi snímky - podezrele mista v kvalite video zaznamu

In [ ]:
(df_avg - df_avg.shift())[['FrameTimestamp_us', 'avg_edge_occurence']].describe(percentiles=[.01, .1, .25, .5, .75, .9, .99])

In [ ]:
df_avg['edge_shift'] = df_avg.avg_edge_occurence - df_avg.shift().avg_edge_occurence
px.line(x=df_avg.FrameTimestamp_us, y=df_avg.edge_shift)

In [ ]:
Z předchozího grafu lze vidět, že přechody mezi snímky odpovídají velikosti rozkmitu. Obsahují pouze jednotlivé výkyvy (vzniklé například přeskočením snímku) a nelze nalézt region, který by obsahoval větší počet podezřelých přechodů.

In [ ]:
#### Detekce periody

Pro výpočet byl použit postup počítání přechodů "0" signálu v našem případě použit průměr jako hranice přechodu rozkmitu.

U výpočtu průměrné hladiny rozkmitu bylo zvoleno okno 800 záznamů z videa. Tato hodnota byla vypočtena z předpokladu alespoň 10 cyklů pro určení aktuální frekvence a nejnižší předpokládáne frekvenci 5 Hz.

Jelikož se ukázalo, že u detekce přechodu přes průměr dochází k nepřesnostem, byl použit průměr 10 záznamů pro vyhlazení originálního signálu. 

In [ ]:
df_avg['smoothed_avg_edge_occurence'] = df_avg.rolling(10, center=True).avg_edge_occurence.mean()
df_avg['rolling_avg_edge_occurence'] = df_avg.rolling(800, center=True).avg_edge_occurence.mean()

In [ ]:
df_tmp = df_avg.iloc[10000:150000]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_us'], value_vars=['avg_edge_occurence', 'smoothed_avg_edge_occurence', 'rolling_avg_edge_occurence'])
px.line(df_tmp, x='FrameTimestamp_us', y='value', color='variable')

In [ ]:
df_avg.loc[df_avg.avg_edge_occurence>=df_avg.rolling_avg_edge_occurence, 'polarity'] = 1
df_avg.loc[df_avg.avg_edge_occurence<df_avg.rolling_avg_edge_occurence, 'polarity'] = -1
df_avg['polarity_shift_abs'] = abs(df_avg.polarity-df_avg.polarity.shift())
df_avg['polarity_shift'] = df_avg.polarity-df_avg.polarity.shift()
df_avg.loc[df_avg.smoothed_avg_edge_occurence>=df_avg.rolling_avg_edge_occurence, 'polarity_smoothed'] = 1
df_avg.loc[df_avg.smoothed_avg_edge_occurence<df_avg.rolling_avg_edge_occurence, 'polarity_smoothed'] = -1
df_avg['polarity_smoothed_shift_abs'] = abs(df_avg.polarity_smoothed-df_avg.polarity_smoothed.shift())
df_avg['polarity_smoothed_shift'] = df_avg.polarity_smoothed-df_avg.polarity_smoothed.shift()

In [ ]:
df_tmp = df_avg.iloc[10000:150000]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_us'], value_vars=['avg_edge_occurence', 'rolling_avg_edge_occurence', 'polarity', 'polarity_shift', 'polarity_smoothed', 'polarity_smoothed_shift'])
px.line(df_tmp, x='FrameTimestamp_us', y='value', color='variable')

In [ ]:
df_tmp = df_avg[(df_avg.FrameTimestamp_us >= 244925000) & (df_avg.FrameTimestamp_us <= 245200000)]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_us'], value_vars=['avg_edge_occurence', 'rolling_avg_edge_occurence', 'polarity', 'polarity_shift', 'polarity_smoothed', 'polarity_smoothed_shift'])
px.line(df_tmp, x='FrameTimestamp_us', y='value', color='variable')

In [ ]:
#### Výpočet periody

In [ ]:
df_avg['frames_timediff_us'] = df_avg.FrameTimestamp_us - df_avg.FrameTimestamp_us.shift()
df_avg['periody_start_mark'] = 0
df_avg.loc[df_avg.polarity_smoothed_shift == 2, 'periody_start_mark'] = 1
df_avg['rolling_sum_frames_timediff_us'] = df_avg.rolling(800, center=True).frames_timediff_us.sum()
df_avg['rolling_sum_periody_start_mark'] = df_avg.rolling(800, center=True).periody_start_mark.sum()
df_avg['frequency'] = 1 / (df_avg.rolling_sum_frames_timediff_us / df_avg.rolling_sum_periody_start_mark / 10**6)

In [ ]:
df_avg.iloc[1100:1150,:]

In [ ]:
df_avg.frequency.describe()

In [ ]:
df_tmp = df_avg.iloc[10000:150000]
px.line(df_tmp, x='FrameTimestamp_us', y='frequency')

In [ ]:
px.line(df_avg, x='FrameTimestamp_us', y='frequency')

In [ ]:
##### Několik omezení, se kterým zatím pracujeme:

Při výpočtu průměru se počítá s centrováním plovoucího okna, což nepočítá s real time zpracováním.

